In [ ]:
# 对队员进行了爬取
from lxml import etree
import requests
import os


# 获取队伍信息队员信息
pindex = 1
tindex = 1

#href-队员 返回图片path、身高体重
def playerinfo(href):
    r = requests.get(href)
    print(f'    player{pindex}----{href} : {str(r.status_code)}')
    pindex = pindex + 1
    html = etree.HTML(r.text)
    imghref = html.xpath("//div[@id='info']//img[@itemscope='image']/@src")
    playerheight = html.xpath("//div[@id='info']//span[@itemprop='height']/text()")
    playerweight = html.xpath("//div[@id='info']//span[@itemprop='weight']/text()")
    ret = {
        'pimg' : imghref,
        'pweight' : playerweight,
        'pheight' : playerheight,
    }
    return ret

#href-队伍 
#返回队员信息：名字、href、imgpath、身高体重
def teamplayer(href):
    r = requests.get(href)
    print(f'team{tindex}----{href} : {str(r.status_code)}')
    tindex = tindex + 1
    html = etree.HTML(r.text)
    playerlist = html.xpath("//div[@id='all_roster']//tbody//td[@data-stat='player']/a/text()")
    playerhref = html.xpath("//div[@id='all_roster']//tbody//td[@data-stat='player']/a/@href")
    players = []
    for i in range(0, len(playerlist)):
        pname = playerlist[i]
        pinfo = playerinfo(f'{playerhref[i]}')
        phref = playerhref[i]
        info = {
            'pname' : pname,
            'phref' : phref,
            'pimg' : pinfo['pimg'],
            'pheight' : pinfo['pheight'],
            'pweight' : pinfo['pweight'],
        }
        players.append(info)
        print(f'         {pname} {phref} {pinfo["pimg"]} {pinfo["pheight"]} {pinfo["pweight"]}')
    return players

#返回队伍列表
#名字、city、roster、href
def teaminfo():
    r = requests.get("https://www.basketball-reference.com/leagues/NBA_2021.html")
    html = etree.HTML(r.text)
    teamname = html.xpath('//*[@id="per_game-team"]//tr//a/text()')
    teamhref = html.xpath('//*[@id="per_game-team"]//tr//a/@href')
    teamname[4] = 'Portland Trail Blazers'

    teamlist = []
    for i in range(0, len(teamname)):
        tname = teamname[i].split(' ')[-1]
        tcity = teamname[i].replace(f' {tname}', '')
        tpath = f'https://www.basketball-reference.com{teamhref[i]}'
        teamhref[i] = tpath
        roster = teamplayer(tpath)

        tinfo = {
            'tname' : tname,
            'tcity' : tcity,
            'roster' : roster,
            'thref' : tpath,
        }
        teamlist.append(tinfo)
    return teamlist

teamlist = teaminfo()

In [ ]:
import re
def cleantxt(raw):
	return re.sub("[^a-zA-Z]", "", raw)

In [ ]:
import csv

header = ['tname', 'tcity', 'thref']
tlist = []
for team in teamlist:
    tlist.append((team['tname'], team['tcity'], team['thref']))
with open('teamlist.csv', 'w') as f:
    fwriter = csv.writer(f)
    fwriter.writerow(header)
    fwriter.writerows(tlist)


header = ['pname', 'pimgpath', 'pw', 'ph', 'phref', 'team']
plist = []
for team in teamlist:
    for player in team['roster']:
        player['pimg'] = f"{cleantxt(player['pname'])}.jpg"
        player['pweight'] = str(player['pweight']).replace("['", '').replace("']", '').replace("[", '').replace("]", '')
        player['pheight'] = str(player['pheight']).replace("['", '').replace("']", '').replace("[", '').replace("]", '')
        player['phref'] = f'{player["phref"]}'
        plist.append((player['pname'], player['pimg'], player['pweight'], player['pheight'], player['phref'], team['tname']))
with open('playerlist.csv', 'w') as f:
    fwriter = csv.writer(f)
    fwriter.writerow(header)
    fwriter.writerows(plist)

In [ ]:
for team in teamlist:
    for player in team['roster']:
        url = str(player['pimg'])
        url = url.replace("['", '').replace("']", '').replace("[", '').replace("]", '')
        name = cleantxt(player["pname"])
        if url:
            #os.system(f'wget {url}')
            os.system(f'mv {url.split("/")[-1]} playerimg/{name}.jpg')
            print(f'mv {url.split("/")[-1]} playerimg/{name}.jpg')

In [ ]:
os.system('mv ./sql.py ./playerimg/sql2.py')

In [ ]:
import re
def cleantxt(raw):
	return re.sub("[^a-zA-Z]", "", raw)

In [ ]:
from lxml import etree
with open('h.html', 'r') as f:
    text = f.read()
html = etree.XPath(text)
